In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        #del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


In [4]:
df_list = ['eps1', 'se']
input_df = pd.concat([data[i] for i in df_list], axis = 1)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TimeBlock(nn.Module):
    """
    Ein einzelner Zeitblock, der periodische Muster auf mehreren Skalen analysiert.
    """
    def __init__(self, input_dim, hidden_dim, num_periods):
        super(TimeBlock, self).__init__()
        self.num_periods = num_periods
        self.linear_periods = nn.ModuleList([
            nn.Linear(input_dim, hidden_dim) for _ in range(num_periods)
        ])
        self.linear_combine = nn.Linear(hidden_dim * num_periods, input_dim)
    
    def forward(self, x):
        # x: [Batch, Time, Features]
        batch_size, seq_len, feature_dim = x.size()
        outputs = []
        
        # Zeitliche Perioden transformieren
        for i, linear in enumerate(self.linear_periods):
            period_length = seq_len // (i + 1)
            reshaped = x[:, :period_length * (i + 1), :].view(batch_size, -1, i + 1, feature_dim)
            mean_over_period = reshaped.mean(dim=2)  # Mittelwert über jede Periode
            outputs.append(F.relu(linear(mean_over_period)))  # Transformation
        
        # Kombinieren der Ergebnisse
        combined = torch.cat(outputs, dim=-1)  # [Batch, Time, Hidden_Dim * Num_Periods]
        out = self.linear_combine(combined)
        return out + x  # Residual-Verbindung

class TimesNet(nn.Module):
    """
    TimesNet-Modell für Zeitreihenanalyse.
    """
    def __init__(self, input_dim, hidden_dim, num_blocks, num_periods):
        super(TimesNet, self).__init__()
        self.blocks = nn.ModuleList([
            TimeBlock(input_dim, hidden_dim, num_periods) for _ in range(num_blocks)
        ])
        self.output_layer = nn.Linear(input_dim, input_dim)
    
    def forward(self, x):
        # x: [Batch, Time, Features]
        for block in self.blocks:
            x = block(x)  # Durchlaufen der Zeitblöcke
        out = self.output_layer(x.mean(dim=1))  # Durchschnitt über die Zeitdimension
        return out

# Beispiel: Modell initialisieren und testen
if __name__ == "__main__":
    batch_size = 16
    seq_len = 10  # Anzahl der Zeitschritte
    input_dim = 60  # Anzahl der Merkmale
    hidden_dim = 64
    num_blocks = 3
    num_periods = 4

    model = TimesNet(input_dim=input_dim, hidden_dim=hidden_dim, num_blocks=num_blocks, num_periods=num_periods)
    x = input_df
    output = model(x)

    print("Eingabeform:", x.shape)
    print("Ausgabeform:", output.shape)


TypeError: 'int' object is not callable